In [23]:
############# TRATATIVA E NORMALIZAÇÃO DOS DADOS ######################
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV 
import joblib 
import os 

df_obesity = pd.read_csv("C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\data\\Obesity.csv")


# Passos de exploração baixo estão descrito no Exploracao.ipynb
novos_nomes = {
    'Gender': 'Gênero',
    'Age': 'Idade',
    'Height': 'Altura',
    'Weight': 'Peso',
    'family_history': 'Histórico_Familiar_Obesidade',
    'FAVC': 'Frequencia_Consumo_Alimento_Calorico',
    'FCVC': 'Frequencia_Consumo_Vegetais',
    'NCP': 'Numero_Refeicoes_Principais',
    'CAEC': 'Consumo_Alimento_Entre_Refeicoes',
    'SMOKE': 'Fumante',
    'CH2O': 'Consumo_Agua',
    'SCC': 'Monitoramento_Calorico',
    'FAF': 'Frequencia_Atividade_Fisica',
    'TUE': 'Tempo_Uso_Tecnologia',
    'CALC': 'Consumo_Alcool',
    'MTRANS': 'Meio_Transporte',
    'Obesity': 'Status_Obesidade'  
}

df_obesity = df_obesity.rename(columns=novos_nomes)

#Colunas de sim e nao para 1 e 0
colunas_sim_nao = ['Histórico_Familiar_Obesidade', 'Frequencia_Consumo_Alimento_Calorico', 'Fumante', 'Monitoramento_Calorico']

dict_sin_nao = {'yes': 1, 'no': 0}

df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)




#coluna gênero para 0 e 1

coluna_genero = ['Gênero']
dic_genero = {'Female' : 0,'Male': 1}

df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)



#colunas com classificacao de 0 a 3 para valores numericos
colunas_class_0_3 = ['Consumo_Alcool','Consumo_Alimento_Entre_Refeicoes']
dic_class_0_3 = {'no': 0,'Sometimes': 1,'Frequently': 2, 'Always': 3}

df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_class_0_3).astype(int)


### Colunas onde ponto flutuante está incorreto, pegando o primeiro digito

colunas_primeiro_digito = [ 'Tempo_Uso_Tecnologia','Frequencia_Atividade_Fisica',
'Consumo_Agua','Numero_Refeicoes_Principais','Frequencia_Consumo_Vegetais']

for coluna in colunas_primeiro_digito:
    df_obesity[coluna] = df_obesity[coluna].astype(str).str[0]

    df_obesity[coluna] = df_obesity[coluna].astype(int)

# Coluna de idade arredondado
df_obesity['Idade'] = df_obesity['Idade'].astype(int)
df_obesity['Peso'] = df_obesity['Peso'].astype(float).round(2)

#IREMOS TESTAR 3 ABORDAGENS 

C:\Users\Igor\AppData\Local\Temp\ipykernel_15340\1602525064.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)
C:\Users\Igor\AppData\Local\Temp\ipykernel_15340\1602525064.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)


In [24]:

# 1- CLASSIFICANDO OBESIDADE EM 2 NIVEIS: OBESO E NAO OBESO

#Coluna de classificacao de obesiade para valor numerico

df_obesity_biaria = df_obesity

def normalize_obesity(level):
    if level in ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II']:
        return 0  # Não obeso
    else:
        return 1  # Obeso

df_obesity_biaria["Status_Obesidade"] = df_obesity_biaria["Status_Obesidade"].apply(normalize_obesity)

df_obesity_biaria.head()

,Gênero,Idade,Altura,Peso,Histórico_Familiar_Obesidade,Frequencia_Consumo_Alimento_Calorico,Frequencia_Consumo_Vegetais,Numero_Refeicoes_Principais,Consumo_Alimento_Entre_Refeicoes,Fumante,Consumo_Agua,Monitoramento_Calorico,Frequencia_Atividade_Fisica,Tempo_Uso_Tecnologia,Consumo_Alcool,Meio_Transporte,Status_Obesidade
0,0,21,1.62,64.0,1,0,2,3,Sometimes,0,2,0,0,1,no,Public_Transportation,0
1,0,21,1.52,56.0,1,0,3,3,Sometimes,1,3,1,3,0,Sometimes,Public_Transportation,0
2,1,23,1.80,77.0,1,0,2,3,Sometimes,0,2,0,2,1,Frequently,Public_Transportation,0
3,1,27,1.80,87.0,0,0,3,3,Sometimes,0,2,0,2,0,Frequently,Walking,0
4,1,22,1.78,89.8,0,0,2,1,Sometimes,0,2,0,0,0,Sometimes,Public_Transportation,0
